# Autoencoder Training Notebook
In this Jupyter notebook you'll process the data you collected from your freezer and use to train an autoencoder machine learning model that will be able to detect anomalies in the operation of your freezer. Going through this notebook should take approximatly 30 minutes and it's best to do it in one sitting. We use Python and a few machine learning specific libraries, you won't have to write any code, just follow the steps and run the code blocks. If you're not familiar with Jupyter notebooks here is a useful tutorial for getting started.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

## Download your data
First you'll need to download your temperature data from Adafruit.io. Adafruit put together a great guide on how to do this [here](https://learn.adafruit.com/adafruit-io-basics-feeds/downloading-feed-data). Make sure to download the data as a CSV and move into the same folder as this notebook.

In [3]:
# Path to the dataset from Adafruit.io
# Change <PATH-TO-DATASET> to the file name of the Adafruit.io data you downloaded.
dataset_path = './dataset/raw/known_good.csv'

# Load the dataset using pandas
df = pd.read_csv(dataset_path, usecols=[1])

df

,-19.5625
0,-19.6250
1,-19.6250
2,-19.6875
3,-19.6875
4,-19.7500
...,...
228511,-18.2500
228512,-18.2500
228513,-18.2500
228514,-18.1875


## Processing your data
Next you'll restructure the data set so that each line represtened the time period you want to look at. An hour seems to be long enough to account for any normal irregualties in the temperature.

In [4]:
# The sample rate should match the code on your temperature monitor

def hours_to_seconds(hours):
    return hours * 3600

sample_rate = 0.2   # Hz (samples per second)
sample_time = 1     # Hours
datapoints_per_row = sample_rate * hours_to_seconds(sample_time)
start_time = 0

x_train = pd.DataFrame()
arr = []
for i, temp in df.iterrows():
    if i % datapoints_per_row == 0 and i != start_time:
        sample = pd.DataFrame(data=arr).T
        x_train = x_train.append(sample, ignore_index=True)
        arr = []
    arr.append(df.values[i])
raw_data = x_train.values

When you train a machine learning model you're going to want to make sure that it works before putting out in the world. To do that you reserve some amount of the data for testing after you train the model. You'll also use validation set during training, but we'll split that one out of the training data later.

In [5]:
train_data, test_data = train_test_split(
    raw_data, test_size=0.2
)

In [6]:
# You can use this data later to verify that your microconroller code is working correctly
export_data = np.array(test_data)
np.savetxt('arduino_test.csv', export_data, delimiter=',')

Nueral networks tend to work best when the values of the data is on a similar scale. It's good practice to normalize your data, in this case you'll normalize your data to values between 0 and 1.

In [7]:
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

Lets graph a normal hour of temperature data. Try changing the index value of "train_data" to see how different parts of the dataset look.

In [ ]:
plt.grid()
plt.plot(np.arange(720), train_data[0])
plt.title("Normal Temps")
plt.show()

## Training your model
In this section you'll define the model architecture and train the model. Then you'll evaulate how the model is performing. We won't go into detial of how Tensor Flow works here, but there further reading the readme has great content about this.

In [ ]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])
    
    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(720, activation="sigmoid")])
    
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mae')

In [ ]:
history = autoencoder.fit(train_data, train_data, 
          epochs=60, 
          batch_size=16,
          validation_split = 0.1,
          shuffle=True)

Now that you've trained you model you can look at the ending loss and validation loss, you want them both to be pretty small, less than 0.05, and close together in value. We an also see in the next graph how the model progressed in training.

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

In [ ]:
encoded_imgs = autoencoder.encoder(train_data).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

plt.plot(train_data[0],'b')
plt.plot(decoded_imgs[0],'r')
plt.fill_between(np.arange(720), decoded_imgs[0], train_data[0], color='lightcoral' )
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

In [ ]:
encoded_imgs = autoencoder.encoder(test_data).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

plt.plot(test_data[0],'b')
plt.plot(decoded_imgs[0],'r')
plt.fill_between(np.arange(720), decoded_imgs[0], test_data[0], color='lightcoral' )
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

In [ ]:
reconstructions = autoencoder.predict(train_data)
train_loss = tf.keras.losses.mae(reconstructions, train_data)

plt.hist(train_loss, bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

## Testing your model
Finally we want to check how accurate the model is with data it's never seen before. First we'll print the metrics from the training set and then we can compare the metrics from the test set. If the metrics look close we know our model has generlized well enough to work.

In [ ]:
def predict(model, data, threshold):
  reconstructions = model(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, preds)))
  print("Precision = {}".format(precision_score(labels, preds)))
  print("Recall = {}".format(recall_score(labels, preds)))

In [ ]:
preds = predict(autoencoder, train_data, threshold)
labels = np.ones(preds.shape[0])
print_stats(preds, labels)

preds = predict(autoencoder, test_data, threshold)
labels = np.ones(preds.shape[0])
print_stats(preds, labels)

## Exporting your model

In [ ]:
print('const int input_size = {};'.format(train_data.shape[1]))
print('const float threshold = {};'.format(threshold))
print('const float min_val = {};'.format(min_val))
print('const float max_val = {};'.format(max_val))

In [ ]:
# Convert the model to a TF Lite flat buffer.
converter = tf.lite.TFLiteConverter.from_keras_model(autoencoder)
# We optimize the model in order to save room on the microcontroller. Optimizing the model will have a slight affect on the results, but for this use case it won't be a problem.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)

## Anomaly Testing

In [ ]:
# Path to the anomaly dataset from Adafruit.io
# Change <ANOMALY-DATASET-NAME> to the file name of the Adafruit.io data you downloaded.
# dataset_path = './dataset/raw/known_anomaly.csv'
dataset_path = './dataset/raw/sept_test.csv'

# Load the dataset using pandas
df_anomaly = pd.read_csv(dataset_path, usecols=[1])

df_anomaly

In [ ]:
# Resort the data set into rows of 1 hour cycles
start_time = 0
samples_per_file = 720

x_train = pd.DataFrame()
arr = []
for i, temp in df_anomaly.iterrows():
    if i % datapoints_per_row == 0 and i != start_time:
        sample = pd.DataFrame(data=arr).T
        x_train = x_train.append(sample, ignore_index=True)
        arr = []
    arr.append(df_anomaly.values[i])
anomaly_data = x_train.values

In [ ]:
anomaly_data = (anomaly_data - min_val) / (max_val - min_val)

anomaly_data = tf.cast(anomaly_data, tf.float32)

In [ ]:
encoded_imgs = autoencoder.encoder(anomaly_data).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

plt.plot(anomaly_data[2],'b')
plt.plot(decoded_imgs[2],'r')
plt.fill_between(np.arange(720), decoded_imgs[2], anomaly_data[2], color='lightcoral' )
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

In [ ]:
reconstructions = autoencoder.predict(test_data)
test_loss = tf.keras.losses.mae(reconstructions, test_data)

plt.hist(test_loss, bins=50)
plt.xlabel("Test loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
preds = predict(autoencoder, anomaly_data, threshold)
print_stats(preds, test_labels)
preds

In [ ]:
interpreter = tf.lite.Interpreter(model_path='./model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(train_data[1])
# print([train_data[0]].shape)
interpreter.set_tensor(input_details[0]['index'], [input_data])

interpreter.invoke()

tflite_results = interpreter.get_tensor(output_details[0]['index'])

loss = tf.keras.losses.mae(tflite_results, train_data[1])
print(loss)